In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [34]:
import cv2
import networkx as nx
import numpy as np
import os
import re
import sys

from models import ChopinNet
from models import BachNet
from utils import display_utils
from utils import preprocessing_utils
from utils import graph_utils
from utils import prediction_utils

In [3]:
receptive_field_shape = (23, 23)
bach = BachNet.BachNet()

In [5]:
bach.build(23, 23, 1)
bach.load_model('models/saved_model/Bach/model.h5')

images_to_train = file("images_to_train.txt", "r")

image_data = dict()

for line in images_to_train:
    
    f_name, image_path, gt_path = line.split()
    
    sys.stdout.write("\rLoading Image: {}".format(f_name))
    
    foldername = os.path.join("data/train/chopin", f_name)
    
    img, gt, seeds = preprocessing_utils.load_image(foldername,
                                                    f_name,
                                                    image_path,
                                                    gt_path)
    
    bps = prediction_utils.boundary_probabilities(bach, img)
    
    image_data[f_name] = img, bps, gt, seeds
    
sys.stdout.write("Done")

Could not find model. Creating a new one.
Loading Image: slice_39data/train/input/slice_39_gt.png data/train/chopin/slice_39
Loading Image: slice_55data/train/input/slice_55_gt.png data/train/chopin/slice_55
Done

In [25]:
chopin = ChopinNet.Chopin()
chopin.build(receptive_field_shape)
chopin.initialize_session()
chopin.load_model("models/saved_model/Chopin/model.ckpt")

INFO:tensorflow:Restoring parameters from models/saved_model/Chopin/model.ckpt


In [28]:
for name, (img, bps, gt, seeds) in image_data.iteritems():
    print(name)
    
    I_a = np.stack((img, bps), axis=2)
    I_a = preprocessing_utils.pad_for_window(I_a,
                               chopin.receptive_field_shape[0],
                               chopin.receptive_field_shape[1])
    
    graph = graph_utils.prims_initialize(img)
    
    msf = prediction_utils.minimum_spanning_forest(chopin, I_a, graph, seeds)

    segmentations = display_utils.assignments(img, msf, seeds)
    
    assignments = nx.get_node_attributes(msf, 'seed')

slice_55
Starting gradient segmentation...
Segmentation done: 310.444661s


NameError: name 'display_utils' is not defined

In [30]:
segmentations = display_utils.assignments(img, msf, seeds)

In [35]:
assignments = nx.get_node_attributes(msf, 'seed')

In [36]:
assignments

{(53, 53): (64, 56),
 (57, 50): (64, 56),
 (39, 70): (64, 56),
 (43, 3): (26, 29),
 (63, 76): (64, 56),
 (29, 44): (26, 29),
 (33, 41): (26, 29),
 (90, 42): (64, 56),
 (80, 12): (64, 56),
 (9, 0): (26, 29),
 (86, 22): (64, 56),
 (52, 98): (52, 93),
 (15, 30): (26, 29),
 (19, 91): (64, 56),
 (76, 88): (64, 56),
 (42, 88): (64, 56),
 (5, 84): (26, 29),
 (62, 31): (64, 56),
 (66, 66): (64, 56),
 (52, 17): (26, 29),
 (8, 87): (64, 56),
 (28, 10): (26, 29),
 (85, 87): (64, 56),
 (32, 77): (64, 56),
 (14, 77): (64, 56),
 (75, 97): (52, 93),
 (95, 98): (93, 93),
 (8, 38): (26, 29),
 (65, 11): (64, 56),
 (85, 56): (64, 56),
 (41, 34): (26, 29),
 (61, 63): (64, 56),
 (88, 67): (64, 56),
 (17, 57): (26, 29),
 (37, 54): (64, 56),
 (94, 57): (64, 56),
 (98, 36): (64, 56),
 (13, 13): (26, 29),
 (74, 13): (64, 56),
 (3, 11): (26, 29),
 (97, 17): (64, 56),
 (23, 84): (64, 56),
 (50, 18): (26, 29),
 (70, 81): (64, 56),
 (40, 4): (26, 29),
 (2, 78): (26, 29),
 (46, 30): (26, 29),
 (12, 90): (64, 56),
 

{(4, 92): 0, (93, 93): 1, (26, 29): 2, (64, 56): 3, (3, 94): 4, (52, 93): 5, (89, 97): 6, (2, 98): 7, (97, 93): 8}
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(52, 93)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(52, 93)
(93, 93)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(26, 29)
(26, 29)
(64, 56)
(26, 29)
(52, 93)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(26, 29)
(26, 29)
(64, 56)
(26, 29)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(52, 93)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(26

(26, 29)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(26, 29)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(26, 29)
(26, 29)
(64, 56)
(26, 29)
(26, 29)
(26, 29)
(52, 93)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(52, 93)
(52, 93)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(64, 56)
(52, 93)
(26, 29)
(52, 93)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(26, 29)
(64, 56)
(26, 29)
(26, 29)
(26, 29)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(26, 29)
(26, 29)
(26, 29)
(64, 56)
(26, 29)
(26, 29)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(

(26, 29)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(26, 29)
(64, 56)
(26, 29)
(64, 56)
(52, 93)
(64, 56)
(64, 56)
(52, 93)
(26, 29)
(64, 56)
(26, 29)
(26, 29)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(52, 93)
(26, 29)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(52, 93)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(52, 93)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(26, 29)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(52, 93)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(64, 56)
(26, 29)
(26, 29)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(26, 29)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(64, 56)
(52, 93)
(26, 29)
(64, 56)
(64, 56)
(26, 29)
(26, 29)
(

In [43]:
assignments

{(53, 53): (64, 56),
 (57, 50): (64, 56),
 (39, 70): (64, 56),
 (43, 3): (26, 29),
 (63, 76): (64, 56),
 (29, 44): (26, 29),
 (33, 41): (26, 29),
 (90, 42): (64, 56),
 (80, 12): (64, 56),
 (9, 0): (26, 29),
 (86, 22): (64, 56),
 (52, 98): (52, 93),
 (15, 30): (26, 29),
 (19, 91): (64, 56),
 (76, 88): (64, 56),
 (42, 88): (64, 56),
 (5, 84): (26, 29),
 (62, 31): (64, 56),
 (66, 66): (64, 56),
 (52, 17): (26, 29),
 (8, 87): (64, 56),
 (28, 10): (26, 29),
 (85, 87): (64, 56),
 (32, 77): (64, 56),
 (14, 77): (64, 56),
 (75, 97): (52, 93),
 (95, 98): (93, 93),
 (8, 38): (26, 29),
 (65, 11): (64, 56),
 (85, 56): (64, 56),
 (41, 34): (26, 29),
 (61, 63): (64, 56),
 (88, 67): (64, 56),
 (17, 57): (26, 29),
 (37, 54): (64, 56),
 (94, 57): (64, 56),
 (98, 36): (64, 56),
 (13, 13): (26, 29),
 (74, 13): (64, 56),
 (3, 11): (26, 29),
 (97, 17): (64, 56),
 (23, 84): (64, 56),
 (50, 18): (26, 29),
 (70, 81): (64, 56),
 (40, 4): (26, 29),
 (2, 78): (26, 29),
 (46, 30): (26, 29),
 (12, 90): (64, 56),
 

In [49]:
segmentation.shape

(100, 100)

In [50]:
segmentation = np.zeros((segmentations.shape[0], segmentations.shape[1]))

for pixel, seg_id in assignments.iteritems():
    segmentation[pixel[0], pixel[1]] = map_coo_to_id[assignments[pixel]]

In [52]:
gt

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ..., 
       [  0,   0,   0, ..., 255,   0,   0],
       [  0,   0,   0, ..., 255, 255,   0],
       [  0,   0,   0, ...,   0, 255, 255]], dtype=uint8)

In [51]:
segmentation

array([[ 2.,  2.,  2., ...,  7.,  7.,  7.],
       [ 2.,  2.,  2., ...,  7.,  7.,  7.],
       [ 2.,  2.,  2., ...,  7.,  7.,  7.],
       ..., 
       [ 3.,  3.,  3., ...,  1.,  1.,  1.],
       [ 3.,  3.,  3., ...,  1.,  1.,  1.],
       [ 3.,  3.,  3., ...,  1.,  1.,  1.]])

In [ ]:
def map_to_seg_ids(pixel_labes):
    """Maps the pixel coordinates to segmentation ids.
    
    This function is to map the pixel ids to a segment id.  This
    is because in order to test using the segmentation metrics,
    the segment ids are required. 
    
    Args:
        pixel_labels: A Dictionary in which the keys are the pixels, 
        and the values are the pixel coordinates of the assigned segment.
        
    Returns:
        A `numpy` array in which the elements are the segment ids.
    
    """

    seed_coos = set(assignments.values())

    # To know the number of segments, count the number of unique segment ids 
    # in the assignment dictionary.
    n_segments = len(seed_coos)

    # A dictionary will be used to map a pixel coordinates to the segmentation id.
    map_coo_to_id = dict()

    # Now create the mapping from pixel coos to semgent ids.
    for segment_id, coo in enumerate(seed_coos):
        map_coo_to_id[coo] = segment_id
        
    segmentation = np.zeros((segmentations.shape[0], segmentations.shape[1]))

    for pixel, seg_id in assignments.iteritems():
        segmentation[pixel[0], pixel[1]] = map_coo_to_id[assignments[pixel]]

In [ ]:
# Get neuron ids from Cremi data.
with h5py.File(hfile, "r") as hdf:
    print ("Initializing...")
    labels = hdf['volumes']['labels']['neuron_ids'][:]
    print ("Done!")